In [31]:
!pip install requests
!pip install pandas
!pip install mp-api
!pip install requests-html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 443.0 kB/s eta 0:00:000:00:01 eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 216.8 kB/s eta 0:00:001m257.8 kB/s eta 0:00:01


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 506.7 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 266.6 kB/s eta 0:00:00m eta 0:00:010:00:01
Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
Using legacy 'setup.py install' for parse, since package 'wheel' is not installed.
  Running setup.py install for parse ... done
  Running setup.py install for bs4 ... done


In [2]:
import csv
import json
import os
import pandas as pd
from mp_api.client import MPRester
from tqdm.notebook import tqdm
from ast import literal_eval
import math
from concurrent.futures import ThreadPoolExecutor
import requests
from requests_html import HTMLSession

In [4]:
ROOT_DIR = os.path.abspath("")
TRAINING_CSV_FILE = os.path.join(ROOT_DIR, "Vacancyenergydata.csv")
API_KEY = "3XX6x9Nd3cKWf9Q4wpaio64CBHFp2uDq"
TRAINING_DATA = []
mpr = MPRester(API_KEY)

print(TRAINING_CSV_FILE)

/home/murimikelvin/ML-perovskites-project/Vacancyenergydata.csv


## Step1:Collecting data.

#### Load Training data.

In [5]:
df = pd.read_csv(TRAINING_CSV_FILE)
print(df.columns)
df = df[['Chemical formula', 'Vacancy energy [eV/O atom]']]
new_names = {
    'Chemical formula': 'chem_formula',
    'Vacancy energy [eV/O atom]': 'e_ovac'
}
df.rename(columns=new_names, inplace=True)
df = df.to_dict("records")
df[:10]


Index(['Chemical formula', 'A', 'B', 'In literature', 'Valence A', 'Valence B',
       'Radius A [ang]', 'Radius B [ang]', 'Lowest distortion',
       'Formation energy [eV/atom]', 'Stability [eV/atom]',
       'Magnetic moment [mu_B]', 'Volume per atom [A^3/atom]', 'Band gap [eV]',
       'a [ang]', 'b [ang]', 'c [ang]', 'alpha [deg]', 'beta [deg]',
       'gamma [deg]', 'Vacancy energy [eV/O atom]'],
      dtype='object')


[{'chem_formula': 'Ac2O3', 'e_ovac': '3.150'},
 {'chem_formula': 'AcAgO3', 'e_ovac': '0.817'},
 {'chem_formula': 'AcAlO3', 'e_ovac': '6.695'},
 {'chem_formula': 'AcAsO3', 'e_ovac': '3.634'},
 {'chem_formula': 'AcAuO3', 'e_ovac': '0.807'},
 {'chem_formula': 'AcBO3', 'e_ovac': '4.163'},
 {'chem_formula': 'AcBaO3', 'e_ovac': '-1.771'},
 {'chem_formula': 'AcBeO3', 'e_ovac': '-0.080'},
 {'chem_formula': 'AcBiO3', 'e_ovac': '2.653'},
 {'chem_formula': 'AcCaO3', 'e_ovac': '-1.025'}]

#### Loading prediction data

In [36]:
list_of_available_fields = mpr.summary.available_fields

print(dir(mpr.summary.search))

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [8]:

# mpr.materials.available_fields

fields = ['nelements', 'formula_pretty', 'formula_anonymous', 'composition', 'composition_reduced', 'icsd_ids']

docs = mpr.summary.search(elements=["O"], fields=fields)



Retrieving SummaryDoc documents: 100%|████████████████████████████████████████████████████████████████████████| 81887/81887 [01:05<00:00, 1250.78it/s]


In [9]:
print(len(docs))

81887


## Step 2: Data cleaning

#### Cleaning training data

In [8]:
TRAINING_DATA = [m for m in df if m['e_ovac'] != '-']
PREDICTION_DATA = [m for m in df if m['e_ovac'] == '-']



print("Materials in the csv file without E_Ovac: ", len(PREDICTION_DATA))
print("Materials in the csv file with E_Ovac: ", len(TRAINING_DATA))
print("Total materials in the csv file: ", len(df))


Materials in the csv file without E_Ovac:  415
Materials in the csv file with E_Ovac:  4914
Total materials in the csv file:  5329


In [51]:
print(TRAINING_DATA[0])

{'chem_formula': 'Ac2O3', 'e_ovac': '3.150'}


#### Cleaning predicting data

In [11]:
for material in docs:
    print(material)
    break

MPDataDoc<SummaryDoc>
nelements=2,
composition=Comp: Ta4 O12,
composition_reduced=Comp: Ta1 O3,
formula_pretty='TaO3',
formula_anonymous='AB3'

Fields not requested:
['builder_meta', 'nsites', 'elements', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'k_voigt', 'k_reuss', 'k_vrh', 'g_voigt',

In [12]:
pred_data = []
for material in docs:
    if json.loads(material.composition_reduced.to_json())['O'] == 3.0:
        if material.nelements == 3:
            pred_data.append(material)

print(len(pred_data))
    
    

3628


In [13]:
for m in pred_data: print(m.formula_pretty)

H2CO3
CaFeO3
EuFeO3
HfZnO3
MgSnO3
BaSiO3
BaGeO3
HfMgO3
MgIrO3
MgGeO3
CdIrO3
MgRuO3
MgTiO3
HgIrO3
CaRhO3
CaRuO3
ZnOsO3
SnHgO3
HgOsO3
HfHgO3
TiHgO3
MgOsO3
ZrCdO3
TiCdO3
SrIrO3
CdRuO3
BaRhO3
HgRuO3
BaMnO3
CaVO3
MnCdO3
ZrHgO3
CaIrO3
CaHfO3
SiHgO3
CdRhO3
ZnSiO3
HgRhO3
CdSiO3
ZnRhO3
CdSnO3
CdOsO3
ZrZnO3
CaGeO3
MgVO3
CaOsO3
ZnSnO3
CdGeO3
VCdO3
ZnIrO3
TiZnO3
MgSiO3
MgRhO3
ZnGeO3
VZnO3
MnZnO3
VHgO3
SrGeO3
MgZrO3
SrSiO3
SrRhO3
HfCdO3
HgGeO3
BaVO3
SrMnO3
CaMnO3
GePbO3
BaRuO3
TiSnO3
BaZrO3
CeGaO3
Cs3BO3
Cs4ZnO3
K2S2O3
K8NO3
K8PO3
Li8NO3
Li8PO3
MgSiO3
MgSiO3
MgSiO3
MgSiO3
MgSiO3
MgSiO3
KNO3
MgSiO3
Rb8NO3
Rb8PO3
SrZrO3
ZnSiO3
ZnGeO3
Na8PO3
Na8NO3
IClO3
LaYO3
SrNbO3
MnSiO3
SrNbO3
SrSiO3
NdCuO3
SmBO3
Nb27N37O3
NaNO3
Sm4(OF2)3
Sm2CoO3
Na2PrO3
Na2S2O3
Na2UO3
Na2CeO3
Na2N2O3
Na2CeO3
Li2CoO3
Li2CoO3
Li2CoO3
Li2CoO3
Li2CoO3
Li2CoO3
Li2CoO3
Li2CoO3
Li2CoO3
Li2CoO3
Li2MnO3
Li2MnO3
Li2MnO3
Li2MnO3
Li2MnO3
Li2MnO3
NaOsO3
RbOsO3
KMoO3
NaMoO3
KWO3
Tl2SO3
NaVO3
NaCu2O3
NdZrO3
VCrO3
SnPbO3
Na2CO3
Na8BiO3
Li8AsO3
K

In [43]:
materials_data = []
session = HTMLSession()
def get_data(url):
    
    global materials_data
    
    print(f"Getting data from URL: {url}")
    
    r = requests.get(url)
    print(f"Done getting URL [CODE: {r.status_code}]: {url}")
    
    data_list = r.json().get('data')
    materials_data += data_list


num_mate = 19798
limit_p_page = 500
offsets = math.ceil(num_mate / limit_p_page)
base_url = "https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=[%OFFSET%]&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O"
links = []
## Create links to visit
for i in range(offsets):
    url = base_url.replace('[%OFFSET%]', str(i*limit_p_page))
    links.append(url)
    

## Create a threadpool to get the data
# get_data(links[0])
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(get_data, links)

print(len(materials_data))
# for link in links:
#     print(link)
    

Getting data from URL: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=0&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Getting data from URL: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=500&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Getting data from URL: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=1000&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Getting data from URL: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=1500&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Getting data from URL: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id

Done getting URL [CODE: 200]: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=6000&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Getting data from URL: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=11500&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Done getting URL [CODE: 200]: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=8500&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Getting data from URL: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=12000&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Done getting URL [CODE: 200]: https://oqmd.org/oqmdapi/formationenergy?f

Done getting URL [CODE: 200]: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=15500&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Done getting URL [CODE: 200]: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=12500&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Done getting URL [CODE: 200]: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=19000&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Done getting URL [CODE: 200]: https://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,icsd_id,ntypes,%20&limit=500&offset=16000&sort_offset=0&noduplicate=False&desc=False&filter=generic=ABC3%20AND%20ntypes=3%20AND%20element_set=O
Done getting URL [CODE: 200]: https://oqmd.org/oqmdapi/f

In [58]:
## Remove all materials without O3 in their composition
old_number = len(materials_data)

new_materials_data = [m for m in materials_data if 'O3' in m['name']]

print(f"Original materials number: {old_number}")
print(f"Filtered number: {len(new_materials_data)}")
print(f"Filtered out number: {old_number - len(new_materials_data)}")

Original materials number: 19798
Filtered number: 19333
Filtered out number: 465


In [52]:
materials_data[0]

{'name': 'BaInO3', 'entry_id': 349981, 'icsd_id': None, 'ntypes': 3}

In [59]:
## Remove all materials found in the csv that have e_ovac value.
old_number = len(new_materials_data)

csv_whit_e_ovac = [t['chem_formula'] for t in TRAINING_DATA]
print(csv_whit_e_ovac[0])

new_materials_data = [m for m in materials_data if m['name'] not in csv_whit_e_ovac]

print(f"Original materials number: {old_number}")
print(f"Filtered number: {len(new_materials_data)}")
print(f"Filtered out number: {old_number - len(new_materials_data)}")

Ac2O3
Original materials number: 19333
Filtered number: 2176
Filtered out number: 17157


In [ ]:
print(len(set([m['chem_formula'] for m in df if m['e_ovac'] != '-'])))
print(len(set([m['name'] for m in materials_data])))